In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import psycopg2

## Load data from Postgres

In [2]:
# Set postgres credentials
load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db = os.getenv('DB')

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=user,
                        password=password,
                        ipaddress=host,
                        port=port,
                        dbname=db))
cnx = create_engine(postgres_str)

In [4]:
# List DB tables
conn = psycopg2.connect(postgres_str)
cursor = conn.cursor()
cursor.execute("""SELECT relname FROM pg_class WHERE relkind='r'
                  AND relname !~ '^(pg_|sql_)';""") # "rel" is short for relation.

tables_list = [i[0] for i in cursor.fetchall()] # A list() of tables.
#tables_list #check available tables

In [5]:
# Load datasets
shops_df = pd.read_sql_query('''SELECT * FROM shops_en;''', cnx)
item_categories_df = pd.read_sql_query('''SELECT * FROM item_categories_en;''', cnx)
test_df = pd.read_sql_query('''SELECT * FROM test;''', cnx)
sales_train_df = pd.read_sql_query('''SELECT * FROM sales_train;''', cnx)
items_df = pd.read_sql_query('''SELECT * FROM items_en;''', cnx)
cleaning_store_df = pd.read_sql_query('''SELECT * FROM cleaning_store_id;''', cnx)
cleaning_item_category_df = pd.read_sql_query('''SELECT * FROM cleaning_item_category_id;''', cnx)

In [ ]:
#items_df.info()
#sales_train_df.isnull().sum() # check for NaN values

## Preprocessing

In [7]:
# Merge with categories to get the Ids
sales_train_df = pd.merge(sales_train_df, items_df, left_on='item_id', right_on='item_id', how='left')
# Merge with cleaning_item_category_df to get the status of active and non active categories
sales_train_df = pd.merge(sales_train_df, cleaning_item_category_df, left_on='item_category_id', right_on='item_category_id', how='left')
# Merge with cleaning_store_df to get the status of active and non active shops
sales_train_df = pd.merge(sales_train_df, cleaning_store_df, left_on='shop_id', right_on='shop_id', how='left')

In [8]:
# Drop column that contais the items name, category status name, and shop status name
sales_train_df.drop(labels=['item_name', 'category_status','shop_status'], axis=1, inplace=True)

In [10]:
# Set date to YYYY/mm/dd
sales_train_df['date'] = pd.to_datetime(sales_train_df['date'], format='%d.%m.%Y')

# Set property dtypes for all other columns
sales_train_df = sales_train_df.astype({'date_block_num':'Int64',
                                        'shop_id':'Int64',
                                        'item_id':'Int64',
                                        'item_price':'float64',
                                        'item_cnt_day':'Int64',
                                        'item_category_id':'Int64',
                                        'category_status_code':'Int64',
                                        'shop_status_code':'Int64'})

# Set a new DataFrame to work with models
sales_train_clean_df = sales_train_df.copy()

# Filter only active categories and stores
sales_train_clean_df = sales_train_clean_df.query('category_status_code == 1').query('shop_status_code == 1')

# Drop category_status_code and shop_status_code to lightweight the dataframe
sales_train_clean_df.drop(labels=['category_status_code', 'shop_status_code'], axis=1, inplace=True)

# Drop duplicates, just in case
sales_train_clean_df.drop_duplicates(inplace=True)

In [ ]:
# DF with stores and categories only
sales_train_clean_df

## Transforming work DataFrame

In [101]:
sales_train_clean_df_ = sales_train_clean_df.copy()

In [102]:
# Sum the total amount of sold products on a new column called total_item_amount_sold
sales_train_clean_df_['total_item_amount_sold'] = sales_train_clean_df_.apply(lambda x: (x["item_price"] * x["item_cnt_day"]) if x['item_cnt_day'] > 0 else 0, axis=1)

In [103]:
# Get a test sample with only one store #31
shop_31_df = sales_train_clean_df_.query('shop_id == 31').query('item_cnt_day > 0')

# Group by 'date_block_num', 'item_id' and sum the total sold amount for the store 31
shop_31_df_ = shop_31_df.groupby(['date_block_num', 'item_id']).aggregate({'total_item_amount_sold': 'sum','item_cnt_day':'count'}).reset_index().sort_values("date_block_num", ascending = True)

In [104]:
shop_31_df_

,date_block_num,item_id,total_item_amount_sold,item_cnt_day
0,0,27,2499.0,1
2210,0,14733,149.0,1
2211,0,14738,349.0,1
2212,0,14743,298.0,2
2213,0,14748,596.0,4
...,...,...,...,...
96632,33,8757,498.0,2
96633,33,8766,845.0,4
96634,33,8778,676.0,4
96628,33,8736,4196.0,4
